In [ ]:
import mysql.connector
import pandas as pd
from datetime import date

In [ ]:
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = '',
    database = 'pizza_sales'
)
cursor = conn.cursor()

In [ ]:
df = pd.read_excel(r'C:\Users\ACER\OneDrive - mail.unnes.ac.id\Documents\statistika\porto 4 pizza sales\Data Model - Pizza Sales.xlsx')
df.dtypes

order_details_id              int64
order_id                      int64
pizza_id                     object
quantity                      int64
order_date           datetime64[ns]
order_time                   object
unit_price                  float64
total_price                 float64
pizza_size                   object
pizza_category               object
pizza_ingredients            object
pizza_name                   object
dtype: object

In [ ]:
df['order_time'] = pd.to_datetime(df['order_time'], format='%H:%M:%S')
df.dtypes

order_details_id              int64
order_id                      int64
pizza_id                     object
quantity                      int64
order_date           datetime64[ns]
order_time           datetime64[ns]
unit_price                  float64
total_price                 float64
pizza_size                   object
pizza_category               object
pizza_ingredients            object
pizza_name                   object
dtype: object

In [ ]:
df.drop_duplicates()
df['order_details_id'].duplicated().sum()

0

In [5]:
# Mengambil data hanya untuk hari ini
df_today = df[df['order_date'] == date.today()]

In [24]:
df_pizza_order = df_today.drop(['total_price','pizza_size','pizza_category','pizza_ingredients','pizza_name'], axis=1)
df_pizza_name = df_today.groupby(['pizza_id','pizza_size'])['pizza_name'].unique().reset_index()
df_pizza_name = pd.DataFrame(df_pizza_name, columns=['pizza_id', 'pizza_size', 'pizza_name'])
df_pizza = df_today.groupby(['pizza_name','pizza_category'])['pizza_ingredients'].unique().reset_index()
df_pizza_ingred = pd.DataFrame(df_pizza, columns=['pizza_name','pizza_category','pizza_ingredients'])
df_pizza_category = df_today.groupby('pizza_name')['pizza_category'].unique().reset_index()
df_pizza_category = pd.DataFrame(df_pizza_category, columns=['pizza_name','pizza_category'])

In [ ]:
# i try to make def insert data filter(it just call data)
def insert_table(data):
    table_name = data.string
    insert_table_query = f"select * from {table_name}"
    cursor.execute(insert_table_query)
    df = cursor.fetchall()
    data_table = data['pizza_id'].isin([row[0] for row in df])

In [1]:
cursor.execute("SELECT pizza_id FROM pizza_name")
pizza_id = cursor.fetchall()
df_pizza_name = df_pizza_name['pizza_id'].isin([row[0] for row in pizza_id])

NameError: name 'cursor' is not defined

In [ ]:
for row in df_pizza_order.itertuples(index = False):
    values = (row.order_details_id,row.order_id,row.pizza_id,row.quantity,row.order_date,row.order_time,row.unit_price)
    insert_query = "INSERT INTO pizza_order (order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price) \
    values (%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(insert_query, values)
    conn.commit()

In [ ]:
for row in df_pizza_name.itertuples():
    cursor.execute('''INSERT INTO pizza_name (pizza_id, pizza_size, pizza_name)
    values (%s,%s,%s)''', (row.pizza_id, row.pizza_size, row.pizza_name))
    conn.commit()

In [ ]:
for row in df_pizza_ingred.itertuples():
    cursor.execute('''INSERT INTO pizza_category (pizza_name, pizza_category, pizza_ingredients)
    values (%s,%s,%s)''', (row.pizza_name, row.pizza_category, row.pizza_ingredients))
    conn.commit()